In [1]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

import sys
from pathlib import Path
sys.path[0] = str(Path(sys.path[0]).parent)

from utils.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 
INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [2]:
### LOAD DATA ###
import json

data = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
topics = [json.loads(ln) for ln in open("../../data/claim_topics.jsonl")]

# kp = [json.loads(ln) for ln in open("../data/keyphrases.jsonl")]
ex_retrieve = [json.loads(ln) for ln in open("../../data/wiki_doc_retrieved_from_op_train.jsonlist")]
conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]

ex_ranked = [json.loads(ln) for ln in open("../../data/selected_evidence.jsonl")]


In [3]:
### SAMPLE OUTPUT ###
import random
_ = random.randint(0, 1000)
sample_ranked = ex_ranked[_]

sample_ranked

{'cid': 'd99pxly',
 'evidence_info': [{'sentence': 'The suicide rates are sky high because there is little to no help whatsoever for these people and a large porportion of our society is denying their needs in the first place .',
   'evidence': ['Suicide rates are generally higher amongst males, rural and regional dwellers, Aboriginal and Torres Strait Islander people.',
    'F. Stephen Bridges asserts that there is "a high incidence in early spring (April and May) and also a low incidence in winter" of suicides rates.',
    'In 2014, suicides rates amongst under-45 men in UK reached a 15-year high of 78% of the total 5,140.'],
   'keyphrase': [[['Torres', 'Strait', 'Islander', 'people'],
     ['Suicide', 'rates']],
    [['a', 'high', 'incidence'], ['suicides', 'rates']],
    [['suicides', 'rates', 'amongst', 'under-45', 'men', 'in', 'UK'],
     ['a', '15-year', 'high']]]},
  {'sentence': 'Kyler Prescott was killed as a result of going to a hospital for help - the cause of death was in

In [5]:
from utils.keyphrase_extraction import exctract_keyphrase
from keyphrase_vectorizers import KeyphraseCountVectorizer
import random
import re

# TODOs: News Data
# TODOs: Ranking
# TODOs: Ranking, Cosine
# TODOs: Research Evidence Retireval: Context Aware, Neural Retrieval 
# TODOs: Stance
# TODOs: Paralellise
# TODOs: Target ADUs: Premises, Claims, discard non-ADUs

#from BERT_adu_classifier import predict
# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)

_ = random.randint(0, 1000)

arg = [data][_]["arguments"]
queries = []
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', arg)

results = []

for i in sentences:
    # TODOs: Extend Keyphrase Model to Select varying length keyphrases
    kp_ = extract_keyphrase(i, n_gram=3, n_kp=3)
    results.append(kp_)

#     query = ", ".join(i for i in kp_)
#     res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

#     results.append({
#         "argument_sentence": i, 
#         "query": query, 
#         "retireved_documents": res,

#     })

# results

# results

ModuleNotFoundError: No module named 'keyphrase_vectorizers'

In [26]:
# Argument Level

results = []

_ = 90
hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

# queries = []
# sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

results = []

# TODOs: Refactor key_BERT as Class for multiple oututs
#kp_ = (exctract_keyphrase(hate_, n_gram=1, n_kp=1), exctract_keyphrase(hate_, n_gram=2, n_kp=1), exctract_keyphrase(hate_, n_gram=3, n_kp=1))

kp_ = (exctract_keyphrase(hate_, n_gram=5, n_kp=1))

query = " ,".join(i for i in kp_)
query

res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

results.append({
    "argument_sentence": hate_, 
    "query": query, 
    "retireved_documents": res,

})

results


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.367s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['‘Assalamu Alaykom:’ peace be upon you',
   'How We Got to Here: A Charlottesville Reading List',
   'De-radicalization can work for former ISIS fighters',
   'How to Get Assyrians to Stay',
   'How to Get Christians to Stay']}]

In [27]:
results = []
res = [i["_source"]["document"]["title"] for i in wiki_db.search(query_=query, k=5)]

results.append({
    "argument_sentence": hate_, 
    "query": query, 
    "retireved_documents": res,

})

results

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_search [status:200 duration:0.249s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['Refuting ISIS',
   'Refuting ISIS',
   'Take Off (2017 film)',
   'Terrorism in India',
   'Tareq Kamleh']}]

In [ ]:
#queries
query = "the executive the presidential system legislative branches partisan unproductive"
res = search_text(wiki_ev, query_=query, k=3)
res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
import re

print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
### TEST SEARCH ###

def search_text(es, query_, k=5):
    results = es.search(
        index = es.elastic_index,
        query = {
            "match": {
                "document.text": query_,
                },
        },
        size=k)

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    return hits

def search_topic(es, topic, k=5):
    results = es.search(
        index = es.elastic_index,
        body= {
            "size": k,
            "query": {
                "match": {
                    "document.title": topic,
        }}})

    hits = results["hits"]["hits"]
    doc_ids = [row['_source']["document"]["id"] for row in hits]

    title = hits[0]["_source"]["document"]["title"]
    text = hits[0]["_source"]["document"]["text"]

    return {
        "title": title,
        "text": text
    }

query = "government emails"
text = search_text(wiki_ev, query_=query, k=10)

print(text)


In [ ]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

# def search_text(db, query_, k=5):
#     results = db.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits